# PRÁCTICA 1

Lucía Pérez González, Manuel Ramallo Blanco, Alexandre Lorenzo Martínez

## 1 - Preprocesado

### 1.1 - Eliminación de duplicados

In [33]:
# Abrir datasets
import pandas as pd

df_vino = pd.read_csv("data/train.csv")

# Eliminación de duplicados, ignorado quality
cols = df_vino.columns.drop('quality')
df_vino = df_vino.drop_duplicates(subset=cols)

### 1.2 - Binarización de la calidad

In [34]:
def clasificar_vino(valor): 
    if valor < 7: 
        return "baja calidad" 
    elif valor >= 7: 
        return "alta calidad" 
df_vino['calidad'] = df_vino['quality'].apply(clasificar_vino) 
df_vino = df_vino.drop(columns=['quality'])

### 1.3 - Gestión de valores atípicos

Eliminamos aquellas filas que no tienen valores para 4 columnas o más:

Aquellas filas que tengan 4 o más valores atípicos también serán descartadas. Para contar cuántos datos atípicos hay por fila, añadimos una nueva columna.

In [35]:
df_vino['num atipicos'] = 0

#### 1.3.1 - Fixed acidity

#### 1.3.2 - Volatile acidity

#### 1.3.3 - Citric acid

#### 1.3.4 - Residual sugar

#### 1.3.5 - Chlorides

#### 1.3.6 - Free sulfur dioxide

#### 1.3.7 - Total sulfur dioxide

#### 1.3.8 - Density

#### 1.3.8 - pH

#### 1.3.9 - Sulphates

#### 1.3.10 - Alcohol

#### 1.3.11 - Eliminación de filas con muchos datos atípicos

In [36]:
# Lista de columnas numéricas (todas menos 'calidad' y 'num atipicos')
cols_diana = [c for c in df_vino.columns if c not in ["calidad", "num atipicos"]]

# Deteccion de valores atipicos

# Diccionario para guardar atípicos: fila -> columnas
#atipicos_por_fila = {}

def detectar_atipicos(df, cols_diana):
    for col in cols_diana:

        # Calculamos IQR y límites
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 6 * IQR
        upper = Q3 + 6 * IQR
        
        # Declaramos condición de atípico y registramos sus posiciones
        cond_iqr = (df[col] < lower) | (df[col] > upper)    #OLLO! Nico recomendara ver os casos 1 a 1 
        cond_neg = df[col] < 0
        cond_atipico = cond_iqr | cond_neg

        #for idx in df_vino[cond_atipico].index:
        #    if idx not in atipicos_por_fila:
        #        atipicos_por_fila[idx] = []
        #    atipicos_por_fila[idx].append(col)

        # Incrementamos el contador de atípicos por fila
        df["num atipicos"] += cond_atipico.astype(int)
        df.loc[cond_atipico, col] = pd.NA

        df["num atipicos"] += df[col].isna().astype(int)
    return df

df= detectar_atipicos(df_vino, cols_diana)
print("\nNúmero de filas con 0, 1, 2, ... atípicos (IQR + negativos):")
print(df_vino["num atipicos"].value_counts().sort_index())


Número de filas con 0, 1, 2, ... atípicos (IQR + negativos):
num atipicos
0    1829
1    1142
2     516
3     184
4      50
5       8
6       3
Name: count, dtype: int64


In [37]:
# Analisis de valores faltantes por columna

for col in cols_diana:
    # Contamos el numero de valores faltantes (NA) en la columna(
    print(f"Columna '{col}' tiene {(df_vino[col].isna().sum()/len(df_vino)*100):.2f}% valores faltantes.")

Columna 'fixed acidity' tiene 6.22% valores faltantes.
Columna 'volatile acidity' tiene 5.81% valores faltantes.
Columna 'citric acid' tiene 6.51% valores faltantes.
Columna 'residual sugar' tiene 5.73% valores faltantes.
Columna 'chlorides' tiene 7.29% valores faltantes.
Columna 'free sulfur dioxide' tiene 5.57% valores faltantes.
Columna 'total sulfur dioxide' tiene 6.24% valores faltantes.
Columna 'density' tiene 6.08% valores faltantes.
Columna 'pH' tiene 6.03% valores faltantes.
Columna 'sulphates' tiene 6.70% valores faltantes.
Columna 'alcohol' tiene 5.71% valores faltantes.


In [38]:
def eliminar_o_imputacion(df, max_atipicos=3):
    # Eliminar filas con 4 o más valores atípicos   
    df = df[df['num atipicos'] < max_atipicos].copy()

    # Imputacion de valores: mediana para cada nulo de cada columna
    cols_diana = [c for c in df.columns if c not in ["calidad", "num atipicos"]]
    for col in cols_diana:
        mediana = df[col].median()
        df[col] = df[col].fillna(mediana)

    return df,mediana

df_vino,mediana = eliminar_o_imputacion(df_vino)
